# US Immigration Data Lake
## Data Engineering Capstone Project
### Goal: To support U.S Customs & Border Protection Department to make better decisions on Immigration Policies

***

### Overview
The purpose of this data engineering capstone project is to give students a chance to combine what they've learned throughout the program. This project will be an important part of learners portfolio that will help to achieve data engineering-related career goals. We could choose to complete the project provided by the Udacity team or define the scope and data ourselves. I took the first approach in building the Data Lake on the data on immigration to the United States provided by Udacity.

### Business Problem

A business consulting firm specialized in data warehouse services through assisting the enterprises with navigating their data needs and creating strategic operational solutions that deliver tangible business results is contacted by U.S Customs & Border Protection Department. Specifically, they want help with the modernization of department's data warehousing infrastructure by improving performance and ease of use for end users, enhancing functionality, decreasing total cost of ownership while making it possible for real-time decision making. In total, the department is asking for a full suite of services includes helping department with data profiling, data standardization, data acquisition, data transformation and integration. 

The U.S. Customs and Border Protection needs help to see what is hidden behind the data flood. As more and more immigrants move to the US, people want quick and reliable ways to access certain information that can help inform their immigration, such as weather of the destination, demographics of destination. And for regulators to keep track of immigrants and their immigration meta data such as visa type, visa expire date, entry method to the US. The consulting firm aim to model and create a brand new analytics solution on top of the state-of-the-art technolgies available to enable department to unleash insights from data then making better decisions on immigration policies for those who came and will be coming in near future to the US.

### Structure of the Project

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

***
## Step 1: Scope the Project and Gather Data
***

*Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc*

### The Scope
We will be implementing a Data Lake in the cloud that will support answering questions through analytics tables and dashboards. Additionally, as a single source-of-truth database is developed, the U.S Customs & Border Protection Department could open the solution through a web API so backend web services could query the warehouse for information relating to international visitors.

### The Data
The I-94 Immigration Dataset, the Global Temperature Dataset, I94_SAS_Labels_Descriptions.SAS and U.S. City Demographic Dataset are used in this work.

### The Architecture
The whole solution is cloud based on top of **Amazon Web Services (AWS)**. First, all the datasets were loaded and preprocessed with **Apache Spark** from our customer's storage (Here we are doing it locally as a demonstration) and stored it in a staging area in **AWS S3 bucket**. Then, it is loaded into a **Amazon Redshift** cluster using an **Apache Airflow** pipeline that transfers and checks the quality of the data to finally provide the department a Data Lake for their convenient analysis. A data visualization layer with Tableau or Microstrategy can be used for daily reports.

![Architecture](images/architecture.png)

The main information and questions the department may want to extract from the Data Lake would be:

- Immigrants by nationality.
- Immigrants by origin.
- Immigrants by airline.
- Correlations between destination in the U.S and the source country.
- Correlations between destination in the U.S and source climates.
- Correlations between immigration by source region, and the source region temperature.
- Correlations between Immigrants demographics, and states visited.

***

## Step 2: Explore and Assess the Data

****

*In order to familiarize ourselves with the data provided by UdacityThe exploratory data analysis (EDA) was performed to check what data would be useful and what preprocessing steps should be taken in order to clean, organize and join the various datasets in a meaningful data model.*

### Immigration Data

For decades, U.S. immigration officers issued the I-94 Form (Arrival/Departure Record) to foreign visitors (e.g., business visitors, tourists and foreign students) who lawfully entered the United States. The I-94 was a small white paper form that a foreign visitor received from cabin crews on arrival flights and from U.S. Customs and Border Protection at the time of entry into the United States. It listed the traveler's immigration category, port of entry, data of entry into the United States, status expiration date and had a unique 11-digit identifying number assigned to it. Its purpose was to record the traveler's lawful admission to the United States.

This is the main dataset and there is a file for each month of the year of 2016 available in the directory ../../data/18-83510-I94-Data-2016/ in the SAS binary database storage format sas7bdat. Combined, the 12 datasets have got more than 40 million rows (40,790,529) and 28 columns. For most of the work, only the month of April of 2016 data is used which has more than three million records (3,096,313).

In [1]:
# Importing the libraries needed in this project
import os
import pandas as pd
from datetime import datetime

In [2]:
%%time
immigration_fname = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
immigration = pd.read_sas(immigration_fname, 'sas7bdat', encoding="ISO-8859-1")

CPU times: user 1min 28s, sys: 22.2 s, total: 1min 50s
Wall time: 25min 3s


In [5]:
immigration.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,NaN,NaN,...,U,NaN,1979.0,10282016,NaN,NaN,NaN,1.897628e+09,NaN,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,...,Y,NaN,1991.0,D/S,M,NaN,NaN,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,...,NaN,M,1961.0,09302016,M,NaN,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,1988.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,2012.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2


**Data Dictionary**: 

The following table describes the dataset below.

| Column Name | Description |
| :--- | :--- |
| CICID* | ID that uniquely identify one record in the dataset |
| I94YR | 4 digit year |
| I94MON | Numeric month |
| I94CIT | 3 digit code of source city for immigration (Born country) |
| I94RES | 3 digit code of source country for immigration (Residence country) |
| I94PORT | Port addmitted through |
| ARRDATE | Arrival date in the USA |
| I94MODE | Mode of transportation (1 = Air; 2 = Sea; 3 = Land; 9 = Not reported) |
| I94ADDR | State of arrival |
| DEPDATE | Departure date from the USA |
| I94BIR | Age of Respondent in Years |
| I94VISA | Visa codes collapsed into three categories: (1 = Business; 2 = Pleasure; 3 = Student) |
| COUNT | Used for summary statistics |
| DTADFILE | Character Date Field |
| VISAPOST | Department of State where where Visa was issued |
| OCCUP | Occupation that will be performed in U.S. |
| ENTDEPA | Arrival Flag. Whether admitted or paroled into the US |
| ENTDEPD | Departure Flag. Whether departed, lost visa, or deceased |
| ENTDEPU | Update Flag. Update of visa, either apprehended, overstayed, or updated to PR |
| MATFLAG | Match flag |
| BIRYEAR | 4 digit year of birth |
| DTADDTO | Character date field to when admitted in the US (allowed to stay until) |
| GENDER | Gender |
| INSNUM | INS number |
| AIRLINE | Airline used to arrive in U.S. |
| ADMNUM | Admission number, should be unique and not nullable |
| FLTNO | Flight number of Airline used to arrive in U.S. |
| VISATYPE | Class of admission legally admitting the non-immigrant to temporarily stay in U.S. |

The immigration dataset is the fact table. So it will be at the center of the star schema model of the Data Lake.

### Global Temperature Data

In this capstone project we will be using only the GlobalLandTemperaturesByCity from [Kaggle](https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data).

In [13]:
temperature_fname = '../../data2/GlobalLandTemperaturesByCity.csv'
world_temperature = pd.read_csv(temperature_fname)

In [14]:
world_temperature.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


**Data Dictionary**: 

Tha dataset has the record of the world's temperature from year 1743 to 2013. However, since the immigration dataset only has data of the US National Tourism Office in the year of 2016, the vast majority of the data here seems not to be suitable. We will aggregate yhe dataset by country, average the temperatures and use this reduced table to join with I94CIT_I94RES.csv  so that we can place t in out COUNTRY dimension table of our data model.

| Column Name | Description |
| :--- | :--- |
| dt | Date in format YYYY-MM-DD |
| AverageTemperature | Average temperature of the city in a given date |
| City | City Name |
| Country | Country Name |
| Latitude | Latitude |
| Longitude | Longitude |

In [15]:
world_temperature = world_temperature.groupby(["Country"]).agg({"AverageTemperature": "mean", "Latitude": "first", "Longitude": "first"}).reset_index()

In [16]:
world_temperature.head()

,Country,AverageTemperature,Latitude,Longitude
0,Afghanistan,13.816497,36.17N,69.61E
1,Albania,15.525828,40.99N,19.17E
2,Algeria,17.763206,36.17N,3.98E
3,Angola,21.759716,12.05S,13.15E
4,Argentina,16.999216,39.38S,62.43W


### Airports Data

airport-codes.csv contains the list of all airport codes. Some of the columns contain attributes identifying airport locations, other codes (IATA, local if exist) that are relevant to identification of an airport. Original source url is http://ourairports.com/data/airports.csv (stored in archive/data.csv).

| Column Name | Description |
| :--- | :--- |
| ident | Unique identifier |
| type | Type of the airport |
| name | Airport Name |
| elevation_ft | Altitude of the airport |
| continent | Continent |
| iso_country | ISO code of the country of the airport |
| iso_region | ISO code for the region of the airport |
| municipality | City where the airport is located |
| gps_code | GPS code of the airport |
| iata_code | IATA code of the airport |
| local_code | Local code of the airport |
| coordinates | GPS coordinates of the airport |

In [17]:
airport = pd.read_csv("airport-codes_csv.csv")

In [18]:
airport.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


### U.S. City Demographic Data

This dataset contains information about the demographics of all US cities and census-designated places with a population greater or equal to 65,000. This data comes from the US Census Bureau's 2015 American Community Survey. The following table describes the dataset below.

| Column Name | Description |
| :--- | :--- |
| City | Name of the city |
| State | US state of the city |
| Median Age | The median of the age of the population |
| Male Population | Number of the male population |
| Female Population | Number of the female population |
| Total Population | Number of the total population |
| Number of Veterans | Number of veterans living in the city |
| Foreign-born | Number of residents of the city that were not born in the city |
| Average Household Size | Average size of the houses in the city |
| State Code | Code of the state of the city |
| Race | Race class |
| Count | Number of individual of each race |

In [19]:
us_cities_demographics = pd.read_csv("us-cities-demographics.csv", sep=";")

In [20]:
us_cities_demographics.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


### Step 3: Define the Data Model

*In this section of the documentation, the process of extract, transform and load of the data from the various datasets is described. As mentioned before, 3 of the 4 data sources provided by the Udacity team: immigration, temperatures and demographics have been used. Also, descriptions are extracted from labels descriptions file I94_SAS_Labels_Descriptions.SAS*

#### 3.1 Conceptual Data Model
*Map out the conceptual data model and explain why you chose that model*

The immigration dataset is the heart of the data model. This table's data comes from the immigration data sets and contains keys that links to the dimension tables. As this represent the facts of what needs to be analysed - U.S Immigrators from the world -, this was transformed to the **fact table IMMIGRATION** as represented in the schema below. This data was given the most of the focus during the modeling phase. The immigration dataset is also the data source for the **ARRIVALDATE dimension table**. Extraction of all the distinct values of the columns arrdate and depdate applied and various functions were applied to store in the table a number of attributes of a particular date: day, month, year, week of year and day of week.

![Data Model](images/data_model.png)

The **DEMOGRAPHICS dimension table** comes from the demographics dataset and links to the immigration fact table at US state level. This dimension would allow analysts to get insights into migration patterns into the US based on demographics as well as overall population of states. Median Age, Male Population, Female Population, Total Population, Number of Veterans, Foreign-born were first aggregated by City using first function, since they are repeated accross the different rows of the same city. Then, the resulting rows were grouped by State applying the sum function in the numeric columns to make a cosolidated total in each U.S State. The column Race needed to be transformed in order to make its different values to become different columns. It was achieved by usig the pivot function of the pyspark package. As a result a final structure was reached where the columns (BlackOrAfricanAmerican, White, ForeignBorn, AmericanIndianAndAlaskaNative, HispanicOrLatino, Asian, NumberVeterans, FemalePopulation, MalePopulation, TotalPopulation) were obtained for each of the states of the U.S. This will prvide information and answers to questions like do populous states attract more visitors on a monthly basis?

The **COUNTRY dimention table** is made up of data from the global land temperatures by city and the immigration datasets. To get to the structure seen in the figure above, the GlobalLandTemperaturesByCity is combined with the code-descriptions found in the file I94_SAS_Labels_Descriptions.SAS for the columns i94cit and i94res shown in the image below. Firstly, the key-value pairs are extracted from the I94_SAS_Labels_Descriptions.SAS and were saved in csv files in the lookup directory. Following the temperature dataset is aggregated by City and then by Country. Finally, these two intermediary results were joined to form the table COUNTRY.

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

To accomplish all the tasks related to the preprocessing of the datasets, the steps can be found in **etl.py** to load, select, clean, transform and store the resultind datasets in a very convenient way. The open-source framework Apache Spark was the main tool in this journey. Spark provides an interface for programming entire clusters with implicit data parallelism and fault tolerance.

All the logic of preprocessing is concentrated here in order to only represent the general steps of the ETL. This notebook here is only for document purposes whereas the actual run of the ETL takes place in the Spark. We will also drop all duplicate columns when we store the processed dataframes in Amazon S3.

##### Immigration and I94Date datasets
We loadi the data from the SAS file. The process is completed when we generate and the store the processed dataframes to a bucket in Amazon S3 as parquet files.

- Load the immigration file into a Spark dataframe. Only useful columns are loaded as we identified them in the EDA phase. 
- Misread fields are converted to the proper cdata types
- The dates in the immigration dataframe are stored in SAS date format, which is a value that represents the number of days between January 1, 1960, and a specified date. The dates are converted in the dataframe to a string date format in the pattern YYYY-MM-DD
- High missing value columns "visapost", "occup", "entdepu" and "insnum" are dropped
- The stay column is created from calculating the difference in days between the departure (depdate) and arrival (arrdate) date of the visitors
- From the date columns arrdate and depdate, a second dataframe ARRIVALDATE is created
- Save the processed IMMIGRATION and ARRIVALDATE dataframes to the Amazon S3 in the parquet format

#### COUNTRIES dataset
We load the data global temperature dataset and I94CIT_I94RES lookup table with Spark. The process is completed when we generate and the store the processed dataframe to a bucket in Amazon S3 as parquet files.

- Load the csv file of the global temperature and I94CIT_I94RES lookup table
- Aggregate of the temperatures dataset by country and rename new columns
- Join the two datasets
- Save the resulting dataset COUNTRY to the staging area in Amazon S3

#### DEMOGRAPHICS dataset
We load the demographics dataset and I94ADDR lookup table with Spark. The process is completed when we generate and store the processed dataframe to a bucket in Amazon S3 as parquet files.

- Load the csv file of the demographics and I94ADDR lookup table
- Aggregate the demographics dataset by state and rename new columns
- Join the two datasets
- Save the resulting dataset DEMOGRAPHICS to the staging area in Amazon S3

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

**The data pipeline is built inside the etl.py file included with this Capstone Project.**

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

**The quality check is built inside the etl.py file included with this Capstone Project.**

The data quality checks ensures that the ETL has created fact and dimension tables with adequate records. We will check the number of records of the data processed with Spark.

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

### Table IMMIGRATION

| Column Name | Description |
| :--- | :--- |
| cicid | Primary Key |
| i94yr | Year |
| i94mon | Month |
| i94cit | 3 digit for the country code where the visitor was born. This is a FK to the COUNTRY dimension table |
| i94res | 3 digit for the country code where the visitor resides in. This is a FK to the COUNTRY dimension table |
| arrdate | Arrival date in the USA. This is a FK to the ARRIVALDATE dimension table |
| i94mode | Mode of transportation (1 = Air; 2 = Sea; 3 = Land; 9 = Not reported). FK to the I94MODE dimension table |
| i94addr | State of arrival. This is a FK to the DEMOGRAPHICS dimension table |
| depdate | Departure date from the USA. This is a FK to the ARRIVALDATE dimension table |
| i94bir | Age of Respondent in Years |
| i94visa | Visa codes collapsed into three categories: (1 = Business; 2 = Pleasure; 3 = Student) |
| gender | Gender |
| airline | Airline used to arrive in U.S. |
| fltno | Flight number of Airline used to arrive in U.S. |
| visatype | Class of admission legally admitting the non-immigrant to temporarily stay in U.S. |
| stay | Number of days in the US |
| state_code | Foreign Key. This is the code of the State as in I94ADDR lookup table |
| country_code | Foreign Key. Country Code |

### Table DEMOGRAPHICS

| Column Name | Description |
| :--- | :--- |
| state_code | Primary Key. This is the code of the State as in I94ADDR lookup table |
| city | Name of the City |
| state | Name of the state |
| black_or_african_american | Number of residents of the race Black Or African American |
| white | Number of residents of the race White |
| foreign_born | Number of residents that born outside th United States |
| american_indian_and_alaska_native | Number of residents of the race American Indian And Alaska Native |
| hispanic_or_latino | Number of residents of the race Hispanic Or Latino |
| asian | Number of residents of the race Asian |
| number_veterans | Number of residents that are war veterans |
| female_population | Number of female population |
| male_Ppopulation | Number of male population |
| total_population | Number total of the population |
| median_age | Median Age of the Immigrants |
| average_household_size | Average size of houses in integer |

### Table COUNTRY

| Column Name | Description |
| :--- | :--- |
| country_code | Country Code. This is the PK. |
| country | Country Name |
| temperature | Average temperature of the country between 1743 and 2013 |
| latitude | GPS Latitude |
| longitude | GPS Longitude |

### Table ARRIVALDATE

| Column Name | Description |
| :--- | :--- |
| arrival_iso_date | Date in the format YYYY-MM-DD. This is the PK. |
| arrival_sasdate | Date in the sas format
| arrival_day | Two digit day |
| arrival_month | Two digit month |
| arrival_year | Four digit for the year |
| arrival_weekofyear | The week of the year |
| date_season | The season during the date |

## Step 5: Complete Project Write Up

### Clearly state the rationale for the choice of tools and technologies for the project.

#### S3
Provides a relatively cheap, easy-to-use with scalability, high availability, security, and performance.

#### Spark
- It's ability to handle multiple file formats with large amounts of data.
- Apache Spark offers a lightning-fast unified analytics engine for big data.
- Spark has easy-to-use APIs for operating on large datasets

### Propose how often the data should be updated and why.

The current I94 immigration data is updated monthly, and hence the data will be updated monthly.

#### Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
     * Deploy this Spark solution on a cluster using AWS (EMR cluster) and use S3 for data and parquet file storage. AWS will easily scale when data increases by 100x. Spark can handle the increase but we would consider increasing the number of nodes in our cluster.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
     * In this scenario, Apache Airflow will be used to schedule and run data pipelines.
 * The database needed to be accessed by 100+ people.
     * In this scenario, we would move our analytics database into Amazon Redshift. The saved parquet files can be bulk copied over to AWS Redshift cluster where it can scale big data requirements and has 'massively parallel' and 'limitless concurrency' for thousands of concurrent queries executed by users.